In [81]:
import duckdb

In [82]:
DATABASE = "../database.db"

In [83]:
WEEK_IN_MS = 1000 * 60 * 60 * 24 * 7
print(WEEK_IN_MS)

604800000


In [87]:
with duckdb.connect(DATABASE) as con:
    con.sql("""
    WITH
        unique_player_game AS (
            SELECT DISTINCT
                season,
                week,
                game_id,
                player_id,
                played_at,
                team,
                opponent,
            FROM player_week_data
        ),
        report_next_game_option AS (
            SELECT
                r.report_id,
                r.player_id,
                r.published_at,
                g.game_id,
                g.played_at,
            FROM report_prediction r
            LEFT JOIN unique_player_game g
                ON r.player_id = g.player_id
            WHERE
                g.played_at > r.published_at
        ),
        report_next_game_closest AS (
            SELECT
                report_id,
                arg_min(game_id, played_at) AS closest_game_id
            FROM report_next_game_option
            GROUP BY report_id
        ),
        report_prev_game_option AS (
            SELECT
                r.report_id,
                r.player_id,
                r.published_at,
                g.game_id,
                g.played_at,
            FROM report_prediction r
            LEFT JOIN unique_player_game g
                ON r.player_id = g.player_id
            WHERE
                r.published_at > g.played_at
        ),
        report_prev_game_closest AS (
            SELECT
                report_id,
                arg_max(game_id, played_at) AS closest_game_id
            FROM report_prev_game_option
            GROUP BY report_id
        ),
        report_with_result AS (
            SELECT
                r.report_id,
                r.player_id,
                d.season,
                d.week,
                d.team,
                d.opponent,
                a.opponent AS prev_opp,
                a.rushing_attempts AS prev_att,
                d.rushing_attempts AS att,
                r.expected_workload_change AS expected,
                CASE
                    WHEN att IS NULL OR prev_att IS NULL THEN 'unknown'
                    WHEN att - prev_att > 5 THEN 'much_higher'
                    WHEN att - prev_att > 4 THEN 'somewhat_higher'
                    WHEN prev_att - att > 5 THEN 'much_lower'
                    WHEN prev_att - att > 4 THEN 'somewhat_lower'
                    ELSE 'similar'
                END AS actual,
            FROM report_prediction r
            LEFT JOIN report_next_game_closest n
                ON r.report_id = n.report_id
            LEFT JOIN player_week_data d
                ON n.closest_game_id = d.game_id
                AND r.player_id = d.player_id
            LEFT JOIN report_prev_game_closest b
                ON r.report_id = b.report_id
            LEFT JOIN player_week_data a
                ON b.closest_game_id = a.game_id
                AND r.player_id = d.player_id
                AND (d.week - a.week) < 2
            WHERE
                a.game_id IS NOT NULL
        ),
        output AS (
            SELECT *
            FROM report_with_result
            WHERE
                week > 1
            ORDER BY
                season DESC,
                week ASC,
                player_id ASC,
                report_id ASC
        )
    SELECT *
    FROM output
    LIMIT 20
    ;
    """).show()

┌───────────────────┬───────────┬────────┬───────┬─────────┬──────────┬──────────┬──────────┬───────┬─────────────────┬─────────────┐
│     report_id     │ player_id │ season │ week  │  team   │ opponent │ prev_opp │ prev_att │  att  │    expected     │   actual    │
│      varchar      │   int64   │ int64  │ int64 │ varchar │ varchar  │ varchar  │  int64   │ int64 │     varchar     │   varchar   │
├───────────────────┼───────────┼────────┼───────┼─────────┼──────────┼──────────┼──────────┼───────┼─────────────────┼─────────────┤
│ rotoballer_201562 │      4147 │   2025 │     2 │ CIN     │ JAX      │ CLE      │        0 │     0 │ much_higher     │ similar     │
│ rotoballer_202245 │      4147 │   2025 │     3 │ CIN     │ MIN      │ JAX      │        0 │     4 │ much_higher     │ similar     │
│ rotoballer_202303 │      4147 │   2025 │     4 │ CIN     │ DEN      │ MIN      │        4 │     1 │ much_higher     │ similar     │
│ rotoballer_202391 │      4147 │   2025 │     4 │ CIN     │ D

In [88]:
with duckdb.connect(DATABASE) as con:
    con.sql("""
    SELECT
        description
    FROM report_prediction
    WHERE report_id = 'rotoballer_210792'
    """).show()

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                                                                                                                                                                                                                                                                                   description                                                    